In [120]:
from BayesNet import BayesNet
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
BATCH_SIZE = 100
TEST_BATCH_SIZE = 5
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        './fmnist', train=True, download=True,
        transform=transforms.ToTensor()),
    batch_size=BATCH_SIZE, shuffle=True, **LOADER_KWARGS)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST(
        './fmnist', train=False, download=True,
        transform=transforms.ToTensor()),
    batch_size=TEST_BATCH_SIZE, shuffle=False, **LOADER_KWARGS)

TRAIN_SIZE = len(train_loader.dataset)
TEST_SIZE = len(test_loader.dataset)
NUM_BATCHES = len(train_loader)
NUM_TEST_BATCHES = len(test_loader)


In [121]:
x = torch.tensor([[1.,1.]])

In [131]:
ostr = "{} {}"
print(ostr.format("nj".ljust(10), "df".ljust(10)))

nj         df        


In [142]:
ostr = "{} {}"
print(ostr.format("NN阿静".ljust(10), "你好".ljust(10)))
print(ostr.format("NN阿静上".ljust(10), "你好".ljust(10)))

NN阿静       你好        
NN阿静上      你好        


In [179]:
class Logger():
    def __init__(self, columns, header=None, just='left' ,print=print):
        self.level = 0
        self.align = {"center": '^', "left": '<', "right": '>'}[just]
        self.max_length = max(list(map(len, columns))+[10])+6
        f = ""
        for _ in range(len(columns)):
            f += "{{:{}{}}}".format(self.align,self.max_length)
        self.format = f.format
        self.pf = print
        if header is not None:
            print(header)
        print(self.format(*columns))
    def pf(self, *args):
        self.pf(self.format(*args))
    def warning():
        pass
    def error():
        pass
    def info():
        pass
l = Logger(["loss1", 'loss1', "ssjf"])
l.pf("lossfgfgf1", 'lofgss1', "sfgdgsjf")

loss1           loss1           ssjf            
lossfgfgf1 lofgss1 sfgdgsjf


In [180]:
"{:0>2}:{:0>2}:{:0>2}".format(1,23,4)

'01:23:04'